# Train NN v0.6: Imporved version of v0.5

- generates N training samples with varied energies $K_{\alpha_1}$, energy splitting and ratios
- trains one MLP regressor for each parameter
- The trained NNs are tested on training set, dev set and test set
- No optimization have been made.

Changes:

- Removed:
    - 
- Changes:
    - gen_set(): 
        - Now norms the spectra to 1 ($\pm$ 0.05 due to scaling and noise)
    - read_set():
        - No longer scales spectra
- New:
    - build_library():
        - Takes number N of spectra to build and generates N spectra, divided in 1e5 subfiles. Then zips them together and deletes subfiles. Saves 80% of space, increases runtime.
    - load_library(): 
        - Load library by unzipping library in subfiles. Reads subfiles and builds data matrix. Deletes Subfiles afterwards.
- Bug fixes
    - Introduced norming spectra to achieve better results

#### Import basic libraries 

In [ ]:
"""
Import functions and tools needed
"""
import os
import plotly.offline as py
import plotly.tools as tls
import matplotlib.pyplot as plt
import matplotlib.mlab as mlab
import numpy as np
import pandas as pd
import matplotlib
import math
import scipy.stats as sc
from scipy.special import wofz
from astropy.modeling.functional_models import Voigt1D 
from scipy.interpolate import UnivariateSpline
from sklearn.neural_network import MLPRegressor as MLPR
from sklearn.neural_network import MLPClassifier as MLPC
import time
from sklearn.externals import joblib
# Redirect stdout
import io 
import sys
# Use subpreocesses
import subprocess
# Use tgz to zip large files into compressed containers
import tarfile
# Use garbage collect to clear unused variables and free memory
import gc
# Control figure size
matplotlib.rcParams['figure.figsize']=(7,5)
py.init_notebook_mode()
# Use plotly as gifure output
def plotly_show():
    fig = plt.gcf()
    py.iplot_mpl(fig)
    plt.grid(True)

#### Define functions

In [ ]:
def calc_area(x, v):
    """
    Calculates the area beneath a spectrum
    """
    width = (x[len(x)-1]-x[0])/(len(x)-1)
    area = 0
    for i in range(len(x)):
        area += v[i] * width
    return area

In [ ]:
def build_library(N, x, noise, Set, verbosity):
    """
    Function wrapped around gen_set() for building the training library. 
        - If N is large, generates multiple msgpack files and then zips them.
    """
    start = time.time()
    label_counter = 1
    if ( N > 1e5 ):
        # Create files with size of 1e5 until all spectra are created
        # Check if file already exists
        if (os.path.isfile("./data/"+Set+".tar.bz2") == True):
            sys.stderr.write('Error: File already exists! \n')
            return 0
        tarfile.open(name = "./data/"+Set+".tar.bz2", mode = "x:bz2")
        tar = tarfile.open(name = "./data/"+Set+".tar.bz2", mode = "w:bz2")
        while ( N >= 1e5 ):
            gen_set(int(1e5), x, noise, "temp_"+str(label_counter)+".sublibrary", verbosity)
            # Zip files to one compressed container and delete single them afterwards
            tar.add("./data/temp_"+str(label_counter)+".sublibrary.spectrum")
            # Remove temporary file
            os.remove("./data/temp_"+str(label_counter)+".sublibrary.spectrum")
            N -= 1e5
            label_counter += 1
        tar.close()
        end = time.time()
        if (verbosity > 0):
            print("+++++++++++++")
            print("Time for generating library: %3.2fs" % (end-start))
            print("+++++++++++++")
    else:
        if (os.path.isfile("./data/"+Set+".tar.bz2") == True):
            sys.stderr.write('Error: File already exists! \n')
            return 0
        tarfile.open(name = "./data/"+Set+".tar.bz2", mode = "x:bz2")
        tar = tarfile.open(name = "./data/"+Set+".tar.bz2", mode = "w:bz2")
        gen_set(int(N), x, noise, "temp_"+str(label_counter)+".sublibrary", verbosity)
        tar.add("./data/temp_"+str(label_counter)+".sublibrary.spectrum")
        os.remove("./data/temp_"+str(label_counter)+".sublibrary.spectrum")
        end = time.time()
        if (verbosity > 0):
            print("+++++++++++++")
            print("Time for generating library: %3.2fs" % (end-start))
            print("+++++++++++++")
    tar.close()

In [ ]:
def gen_set(N, x, noise, Set, verbosity): 
    """
    - Generates a set with N spectra by using the superposition of TWO Voigt profiles with randomly choosen
        parameters 
            gamma1: HWHM of Lorentzian part of Voigt profile 1 
            gamma2: HWHM of Lorentzian part of Voigt profile 2
            sigma1: Standard uncertainty of Gaussian part of Voigt profile 1
            sigma2: Standard uncertainty of Gaussian part of Voigt profile 2
            epsilon: offset to energy E
        The Energy E (K alpha1) is centered around 2014eV 
    """
    if (verbosity > 0):
        start = time.time()
    # Definition of some parameters
    gamma1, sigma1 = 0.345, 0.07
    gamma2, sigma2 = 0.36, 0.08
    labels = np.array(0)
    labels = np.delete(labels, 0)
    # Creating the empty data matrix with dimensions N x d+1
    X = np.zeros((N,len(x)+3))
    runtime = np.array(0)
    runtime = np.delete(runtime, 0)
    """
    For loop loops N times to create N spectra. The single spectrum is evaluate and fitted
    on range x to get equal x values as features (Note: When trained on grid defined by x then
    real data must also be sampled on same grid!). File format:
        File dimensions: N x (2 + d), where d is number of grid points resulting from grid x
        [E dE x1 x2 ... xd]
    """
    if (verbosity > 0):
        loop_time_start = time.time()
    for i in range(N):
        # Generate random distribution (+- 1) around central value of energie E
        E_epsilon = (np.random.random_sample()-0.5)*2
        # Generate random distribution (+- 0.1) around central value of energie dE
        dE_epsilon = (np.random.random_sample()-0.5)*(0.4)
        # Generate random distribution (+- 0.05) around central value of amplitude L1
        dL1 = (np.random.random_sample()-0.5)/15
        # Generate random distribution (+- 0.05) around central value of amplitude L2
        dL2 = (np.random.random_sample()-0.5)/15
        L1 = 0.18 + dL1
        L2 = 0.3 + dL2
        E = 2014 + E_epsilon
        dE = 0.85 + dE_epsilon
        v1 = Voigt1D(x_0=E-dE, amplitude_L=L1, fwhm_L=2*gamma1, fwhm_G=2*sigma1*np.sqrt(2*np.log(2)))
        v2 = Voigt1D(x_0=E, amplitude_L=L2, fwhm_L=2*gamma2, fwhm_G=2*sigma2*np.sqrt(2*np.log(2)))
        # Calculate the ratio of the areas
        R = calc_area(x, v2(x))/calc_area(x,v1(x))
        # Save values in two columns in form of [x,v(x)]
        X[i,0] = E
        X[i,1] = dE
        X[i,2] = R
        if (noise == True):
            # Superpose Voigt profiles and norm them to 1
            append = v1(x)+v2(x)
            # Normalize spectrum to 1
            amp = np.amax(append)
            append  /= amp
            # Apply poisson noise to the data, magnify amplitudes to get poisson function working
            append =np.multiply(append,0.5*1e4)
            append = np.random.poisson(append)
            # Scale down again
            append = np.divide(append, 0.5*1e4)
            # Fill data matrix
            for j in range(len(x)):
                X[i,j+3] = append[j]
        else: 
            # Superpose Voigt profiles and norm them to 1
            append = v1(x)+v2(x)
            # Normalize spectrum to 1
            amp = np.amax(append)
            append  /= amp
            for j in range(len(x)):
                X[i,j+3] = append[j]
        # Runtime control
        if (verbosity > 0):
            if ( i % (N/10) == 0 ):
                loop_time_end = time.time()
                time_diff = loop_time_end-loop_time_start
                runtime = np.append(runtime, time_diff)
                print("Progress: %i/%i, time for loop: %3.2fs" % (i , N, time_diff))
                loop_time_start = time.time()
    # Create pandas dataframe
    df_spectrum = pd.DataFrame(X)
    df_spectrum.to_msgpack('./data/'+Set+'.spectrum')
    # Plot verbosity information about the loop
    if (verbosity > 0):
        end = time.time()
        print("Time for generating the "+Set+" set:", end-start)
        if (verbosity > 1):
            plt.figure()
            plt.title("Runtime for generating the data set "+Set)
            plt.plot(runtime, label="Runtime per N/10 loops")
            plt.grid(True)
            plt.legend()
            plotly_show()

In [ ]:
def load_library(Set, fraction, verbosity):
    """
    Function wrapped around read_set() for loading the training library. 
    """
    start = time.time()
    tar = tarfile.open("./data/"+Set+".tar.bz2", "r:bz2")
    label_counter = 1
    X = np.array(0)
    X = np.delete(X, 0)
    for i in (tar):
        if ( label_counter > fraction ):
            break
        start_loop = time.time()
        tar.extract(i)
        temp = read_set("temp_"+str(label_counter)+".sublibrary")
        N = len(temp) * label_counter
        d = len(temp[0])
        X = np.append(X, temp).reshape(N, d)
        end_loop = time.time()
        if (verbosity > 0):
            print("Time for unpacking: %3.2fs" % (end_loop-start_loop))
        os.remove("./data/temp_"+str(label_counter)+".sublibrary.spectrum")
        label_counter += 1
    tar.close()
    end = time.time()
    if (verbosity > 0):
        print("+++++++++++")
        print("Time for loading the library: %3.2fs" % (end-start))
        print("+++++++++++")
    return X

In [ ]:
def read_set(Set):
    """ 
    Read data and store it in (Nxd) Martix, where N donates 
    the observation (single spectrum) and d the dth feature 
    (datapoint given by choosing x). The data gets fitted 
    by the Splines fit. Also, noise is added when reading the
    data if flag is set.
    """
    start = time.time()
    
    X = pd.read_msgpack('./data/'+Set+'.spectrum') 
    X = X.as_matrix()
    end = time.time()
    print("Time for reading "+Set+" set: %3.2fs" % (end-start))
    return X

In [ ]:
def scale_input(X):
    """
    Feature skaling for NN apporach. It is "highly recommended" to scale input data to either [0:1] or [-1:+1] 
    or standardize it to have mean 0 and variance 1
    Source:
    http://scikit-learn.org/stable/modules/neural_networks_supervised.html#regression
    This function standardizes X 
    """
    from sklearn.preprocessing import StandardScaler  
    scaler = StandardScaler()  
    # Don't cheat - fit only on training data
    N = len(X)
    d = len(X[0]+3)
    y = X[:,:3]
    X = X[:,3:]
    scaler.fit(X)  
    X = scaler.transform(X) 
    X = np.append(y,X, axis=1)
    #for i in range(len(X)):
    #    plt.plot(x,X[i])
    #plt.grid(True)
    #plotly_show()
    return X

In [ ]:
def NN_train(X, model, parameter, scaling, verbosity):
    """
    Trains given model on data X and labels y. Returns trainings score
    """
    start = time.time()
    y = np.ravel(X[:,parameter])
    if (scaling == True):
        X = scale_input(X)
    X = X[:,3:]
    # Set out pipe to catch stdout for getting verbosity output of model.fit
    if (verbosity > 0):
        old_stdout = sys.stdout
        sys.stdout = mystdout = io.StringIO()
    model.fit(X, y)
    # Delete pipe
    if (verbosity > 0):
        sys.stdout = old_stdout
    # Save verbosity output (training loss) in variable
    loss = np.array(0)
    loss = np.delete(loss, 0)
    if (verbosity > 0):
        verbosity_output = mystdout.getvalue()
        verbosity_output = np.array(verbosity_output.split(' '))
        for i in range(4,len(verbosity_output),4):
            if (verbosity_output[i].split('\n')[0] == 'improve'):
                break
            else:
                loss = np.append(loss, float(verbosity_output[i].split('\n')[0]))
    # Save score of training
    score = model.score(X,y)
    end = time.time()
    # Print training statistics depending onb verbosity level
    if (verbosity > 0):
        print("Training time: %3.2f " % (end - start))
        print("Training score: %3.2f " % (score))
        if (verbosity > 1):
            plt.figure()
            plt.title("Training loss per epoch")
            plt.semilogy(loss, label="Loss")
            plt.grid(True)
            plt.legend()
            plotly_show()       
    return score

In [ ]:
def test_envir(X_train, X_dev, X_test, model, param, verbosity, scaling):
    """
    Trains and tests a NN on a given label
    """
    param = 0
    score = NN_train(X_train, model, param, scaling, verbosity = verbosity)
    #Save model via
    joblib.dump(model, './data/1_neural_network.pkl')
    #Load model via
    model2 = joblib.load('./data/1_neural_network.pkl')
    predict_train = model.predict(X_train[:,3:])
    predict_dev = model.predict(X_dev[:,3:])
    plt.plot(X_train[:,param]-predict_train, label=("Train: Loss energy"))
    plt.plot(X_dev[:,0]-predict_dev, label=("Dev: Loss energy"))
    plt.xlabel("datapoint")
    plt.ylabel("Error in arb. units")
    plt.title("Error on true label")
    abserr_train = np.absolute(X_train[:,param]-predict_train)
    abserr_train = np.sum(abserr_train)
    abserr_dev = np.absolute(X_dev[:,param]-predict_dev)
    abserr_dev = np.sum(abserr_dev)
    plt.legend()
    plt.grid(True)
    print("Mean error per prediction in training set in run %3.2f " % (abserr_train/len(X_train)))
    print("Mean error per prediction in dev set in run %3.2f " % (abserr_dev/len(X_dev)))
    plotly_show()    

### Begin of testing the algorithms

In [ ]:
"""
Defining (Hyper)Parameters
"""
exp = 5 # Exponent defining the size of the file
factor = 1
N = int(factor*10**(exp)) # Actual value
noisee = True
comment = "less_features" # Comment for data file name
data_size = str(int(N/1000))+"k_" # Value for labeling the data (in "kilo samples") 
x = np.arange(2008,2018,0.1) # Grid for creating and importing data
# Following the definition of the different 
model_energy = MLPR(max_iter=2000,  activation="relu", verbose = True)
model_splitting = MLPR(max_iter=2000,  activation="relu", verbose = True)
model_ratios = MLPR(max_iter=2000,  activation="relu", verbose = True)

In [ ]:
#build_library(N,x,noise=True,Set="train",verbosity=1)
#build_library(int(N/10),x,noise=True,Set="test",verbosity=1)
#build_library(int(N/10),x,noise=True,Set="dev",verbosity=1)

In [ ]:
Xtrain = load_library(Set="train", fraction=1000, verbosity=0)
Xdev = load_library(Set="dev", fraction=1000, verbosity=0)
Xtest = load_library(Set="test", fraction=1000, verbosity=0)

In [ ]:
test_envir(Xtrain, Xdev, Xtest, model_energy, param = 0, verbosity=2, scaling=False)
test_envir(Xtrain, Xdev, Xtest, model_splitting, param = 1, verbosity=2, scaling=False)
test_envir(Xtrain, Xdev, Xtest, model_ratios, param = 2, verbosity=2, scaling=False)